In [8]:
import numpy as np
import pandas as pd
import pylab as plt

import json, re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from helper import plot_history

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


data_dir = '../nlp_datasets/13_bbc_news_category/'

In [19]:
data = pd.read_csv(data_dir+"bbc-text.csv")
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

Train size: 1780
Test size: 445


0       tv future in the hands of viewers with home th...
1       worldcom boss  left books alone  former worldc...
2       tigers wary of farrell  gamble  leicester say ...
3       yeading face newcastle in fa cup premiership s...
4       ocean s twelve raids box office ocean s twelve...
                              ...                        
1775    china  blocks google news site  china has been...
1776    labour accused of  eu propaganda  a  taxpayer ...
1777    tevez - an argentine in brazil some 65 years a...
1778    water firm suez in argentina row a conflict be...
1779    tv presenter deeley drops cd:uk cat deeley has...
Name: text, Length: 1780, dtype: object

In [ ]:
max_words = 1000
tokenize = Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [14]:
batch_size = 32
epochs = 2
drop_ratio = 0.5
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding

In [15]:
def make_model():
    # Build the model
    model = Sequential()
    model.add(Dense(512, input_shape=(max_words,)))
    model.add(Activation('relu'))
    # model.add(layers.Dropout(drop_ratio))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

model = make_model()
model.summary()
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               512512    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 515,077
Trainable params: 515,077
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/2
51/51 [==============================] - 0s 6ms/step - loss: 0.4338 - accuracy: 0.8777 - val_loss: 0.1251 - val_accuracy: 0.9607
Epoch 2/2
51/51 [==============================] - 0s 3ms/step - loss: 0.0411 - accuracy: 0.9938 - val_loss: 0.1190 - val_accuracy: 0.9551
